In [1]:
%load_ext autoreload
%autoreload 2

import math
import numpy as np
from qiskit import QuantumCircuit
import networkx as nx

Load a graph edge-vertex incidence matrix

In [ ]:
# G = nx.barbell_graph(3, 1) 
m_bridge = 0
G = nx.barbell_graph(3, m_bridge)
# G = nx.octahedral_graph() 
# G = nx.complete_graph(5) 

nb_nodes = G.number_of_nodes()

B = nx.incidence_matrix(G, oriented=True).todense().T
root = 3
b = B[:,root]
deg_root = np.sum(b*b, axis=0)
print('root degree: ',deg_root)

X = np.delete(B, root, 1) # remove col of root
# number of STs
print("number of spanning trees: ",np.round(np.linalg.det(X.T@X)))
# edge-node incidence
nx.draw(G)

n = X.shape[0]
r = X.shape[1]
print("number of edges:", n)
print("rank of X: r = ",r)
k = int(np.ceil(np.log2(n+1)))
print("number control qubits = ", k)

# prepare Clifford loader by normalizing cols
W = X / np.sqrt(np.sum(X*X, axis=0)) 
a = np.linalg.det(W.T@W)
success_proba = a

print('success proba without amplification (theory):', success_proba)


In [ ]:
from initial_states.parallelclifford import ParallelCliffordLoader
from utilities.circuits_amplitude_amplification import amplitude_amplification_circuit

n = X.shape[0]
r = X.shape[1]

# number of control qubits
k = int(np.ceil(np.log2(n+1)))
print("k = ",k, " control qubits")
print("n = ",n, " main qubits")

# basic state appended with k control qubits
psi = ParallelCliffordLoader(W)
psi.name = 'C_X'
circ = QuantumCircuit(k + n)
circ.append(psi, list(range(k, k + n)))

circ_Q = amplitude_amplification_circuit(psi,n,k,r)
circ_Q.draw(output="mpl")

In [4]:
m = 1
# apply Q m times
circ_Q = circ_Q.repeat(m)
circ.append(circ_Q, list(range(0, k + n)))
# Add measurements
meas = QuantumCircuit(n + k, n + k) # n + k qubits, n + k classical bits
meas.barrier(range(n + k)) # the barrier is optional, it is an instruction for the later transpiler
meas.measure(range(k, k + n), range(k, k + n)) # perform the measurement, record it in the classical bits
circ.add_register(meas.cregs[0])
qc = circ.compose(meas)

In [ ]:
from qiskit_aer import Aer
import utilities.plotting_utilities
from qiskit import transpile
rank = r

num_shots = 100000
backend_sim = Aer.get_backend('qasm_simulator')
job_sim = backend_sim.run(
    transpile(qc, backend_sim), 
    shots=num_shots
)
result_sim = job_sim.result()
counts = result_sim.get_counts(qc)

nb_accepted_samples = 0
nb_rejected_samples = 0
for key in counts:
    sample = counts[key]
    spl_sz = utilities.plotting_utilities.sample_size(str(key))
    if spl_sz == rank:
        nb_accepted_samples += int(counts[key])
    else:
        nb_rejected_samples += int(counts[key])

a = np.linalg.det(W.T@W)
success_proba = a

print('success proba without amplification (theory):', success_proba)
print("Now apply Q ", m , " times")
theta_a = math.asin(math.sqrt(a))
success_proba_amplitude_amplified = (math.sin((2*m+1)*theta_a))**2
print('success proba with amplification (theory):', success_proba_amplitude_amplified)
print('success proba with amplification (empirical):', nb_accepted_samples/(nb_accepted_samples + nb_rejected_samples))

### Sanity check 
#### Circuit for $V$ indeed gives a binary decomposition in the control register

Prepare a $\psi$ as element of computational basis

In [ ]:
from qiskit.circuit.library import XGate

psi = np.array([0,1,1,1,1,1])
print(psi)

n = len(psi)
s = psi.sum()
print("s = ", s)

# number of control qubits
k = int(np.ceil(np.log2(n+1)))
print("k = ",k, " control qubits")
print("n = ",n, " main qubits")
w = 2**k - (n+1)
print("w = ", w)
circ_0 = QuantumCircuit(n)
circ_0.name = 'psi'

bin_s = [int(i) for i in list('{0:0b}'.format(s))]
bin_s.reverse()
print("bin(s) = ", bin_s)

for i in range(n):
    if psi[i] == 1:
        circ_0.append(XGate(), [i])
circ_0.draw(output="mpl")

Appending $V$ to $\psi$

In [ ]:
circ = QuantumCircuit(n + k, n + k)
circ.append(circ_0, list(range(k, k + n)))
circ_V = utilities.circuits_amplitude_amplification.CircuitForV(n)
circ_V.name = 'V'
circ.append(circ_V, list(range(0, k + n)))

# Add measurements
meas_0 = QuantumCircuit(n + k, n + k) # n + k qubits, n + k classical bits
meas_0.barrier(range(n + k)) # the barrier is optional, it is an instruction for the later transpiler
meas_0.measure(range(0, k + n), range(0, k + n)) # perform the measurement, record it in the classical bits
# circ_0.add_register(meas_0.cregs[0])
qc_0 = circ.compose(meas_0)
# drawing
qc_0.draw(output="mpl")

Repeat preparation and measurement in computational basis

In [ ]:
from qiskit_aer import Aer

num_shots = 10
backend_sim = Aer.get_backend('qasm_simulator')
job_sim = backend_sim.run(
    transpile(qc_0, backend_sim), 
    shots=num_shots
)
result_sim = job_sim.result()
counts = result_sim.get_counts(qc_0)

nb_res = 0
key_rev = []
for k_id in counts:
    nb_res += 1
    key_rev = k_id
if nb_res > 1:
    print("more than one output: Error")
key = key_rev[::-1]
key_array = np.asarray(list(key), dtype=int)

bin_out = key_array[0:k]
psi_out = key_array[k:(k+n)]

print('|bin_s - bin_out| = ', np.linalg.norm(bin_s - bin_out[0:len(bin_s)]))
print('|psi - psi_out| = ', np.linalg.norm(psi - psi_out))